In [59]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import pyodbc
import seaborn as sns
from scipy import stats
import xlsxwriter
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [ ]:
# price_data = d.fetch_price_data(start_date='2005-01-01')
price_data = pd.read_csv('price_data_18_2_2025.csv')
price_data['Date'] = pd.to_datetime(price_data['Date'])

# Define the rolling window in days (6 months = ~126 trading days)
# rolling_window = 126

# Ensure the DataFrame is sorted by Symbol and Date
price_data = price_data.sort_values(by=['Date'], ascending=True)

# Calculate the 6-month rolling average of Mcap for each symbol
# price_data['Avg_6M_Mcap'] = price_data.groupby('Symbol')['Mcap'].rolling(window=rolling_window).mean().reset_index(level=0, drop=True)

top_1000 =  price_data.groupby('Date', group_keys= False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
price_data_500 = price_data[price_data['Symbol'].isin(top_1000['Symbol'])]

# price_data_500 = price_data[price_data['Symbol'].isin(top_1000['Symbol'])]
df = price_data_500[['Date', 'Symbol', 'Close', 'Mcap']]

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# filtered_stocks = price_data.groupby('Date', group_keys=False).apply(
#     lambda x: x.sort_values(by='Avg_6M_Mcap', ascending=False).iloc[0:501]
# )

In [64]:
mibor = pd.read_csv('India 3-Month Bond Yield Historical Data (1).csv')
mibor['Date'] = pd.to_datetime(mibor['Date'], format='%d-%m-%Y')
mibor['Date'] = mibor['Date'].dt.strftime('%Y-%m-%d')
mibor['Date'] = pd.to_datetime(mibor['Date'])
mibor = mibor.sort_values(by='Date')

nifty500_df = pd.read_csv('Data1.csv')
nifty500_df = nifty500_df[['Date','Nifty 500 ']]
nifty500_df['Date'] = pd.to_datetime(nifty500_df['Date'], format='%d-%m-%Y')
nifty500_df['Date'] = nifty500_df['Date'].dt.strftime('%Y-%m-%d')
nifty500_df['Date'] = pd.to_datetime(nifty500_df['Date'])
nifty500_df.set_index('Date', inplace=True)

In [65]:
# Step 1: Calculate daily returns for each security
df['Security_Daily_Return'] = df.groupby('Symbol')['Close'].pct_change()
# Rolling window for 252 trading days (12 months)
rolling_window = 252
df = df.reset_index()  
df['Avg_12M_Returns'] = df.groupby('Symbol')['Security_Daily_Return'] \
    .apply(lambda x: x.rolling(window=rolling_window).mean()) \
    .reset_index(level=0, drop=True)
nifty500_df['Market_Daily_Return'] = nifty500_df['Nifty 500 '].pct_change()
rolling_window = 252
nifty500_df = nifty500_df.reset_index()  # This will make sure the DataFrame has a unique index
nifty500_df['Avg_12M_Benchmark'] = nifty500_df['Market_Daily_Return'].rolling(window=rolling_window).mean()

# mibor['Mibor_Change'] = mibor['Price'].pct_change()
# rolling_window = 252
# mibor = mibor.reset_index()  # This will make sure the DataFrame has a unique index
# mibor['Avg_12M_Mibor'] = mibor['Mibor_Change'].rolling(window=rolling_window).mean()

# Load and clean the MIBOR data
mibor = mibor.dropna(axis=1, how='all')  # Drop columns that are completely NaN
mibor['Date'] = pd.to_datetime(mibor['Date'], errors='coerce')  # Ensure Date is in datetime format
mibor = mibor.dropna(subset=['Date'])  # Remove rows where Date is NaT

# Compute Mibor Change (Daily Percentage Change)
mibor['Mibor_Change'] = mibor['Price'].pct_change()
# Compute 12-month rolling average (252 trading days)
rolling_window = 252
mibor['Avg_12M_Mibor'] = mibor['Mibor_Change'].rolling(window=rolling_window).mean()
# Reset Index
mibor = mibor.reset_index(drop=True)
# Display claned DataFrame
print(mibor.head())

        Date  Price  Mibor_Change  Avg_12M_Mibor
0 2006-01-02   6.04           NaN            NaN
1 2006-01-03   5.95     -0.014901            NaN
2 2006-01-04   5.95      0.000000            NaN
3 2006-01-05   5.93     -0.003361            NaN
4 2006-01-06   5.97      0.006745            NaN


In [66]:
# Step 1: Merge the security dataframe with the market dataframe on Date
df = pd.merge(df, nifty500_df, on='Date')

# Step 2: Define the rolling beta calculation function
def rolling_beta(group):
    # Ensure proper alignment and rolling calculation for both series
    market_variance = group['Market_Daily_Return'].rolling(window=252).var()
    security_market_covariance = group['Security_Daily_Return'].rolling(window=252).cov(group['Market_Daily_Return'])
    beta = security_market_covariance / market_variance
    return beta
# Step 3: Apply the rolling beta calculation for each security (grouped by Symbol)
df['Beta'] = (
    df.groupby('Symbol', group_keys=False)  # Group by Symbol
    .apply(lambda group: rolling_beta(group))  # Calculate beta
     # Reset index for proper alignment
)
# df.drop(columns='index',inplace=True)

In [ ]:
df = pd.merge_asof(df.sort_values('Date'), 
                   mibor.sort_values('Date'), 
                   on='Date', 
                   direction='backward')
def calculate_alpha(row):
    rs = row['Avg_12M_Returns']
    rf = row['Avg_12M_Mibor']
    rm = row['Avg_12M_Benchmark']
    beta = row['Beta']
    if pd.notna(rs) and pd.notna(rf) and pd.notna(rm) and pd.notna(beta):
        return rs - (rf + beta * (rm - rf))
    return None  # Handle cases where data is missing

df['Alpha'] = df.apply(calculate_alpha, axis=1)
# df = df.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = df[['Date','Symbol','Alpha']]
df.dropna(inplace=True)
# Display the updated DataFrame
df['Date'] = df.groupby('Symbol')['Date'].shift(-1)
df = df.dropna(subset=['Date'])

In [69]:
# df['mean_Alpha'] = df.groupby('Date')['Alpha'].transform('mean')
# df['STD_Alpha'] = df.groupby('Date')['Alpha'].transform('std')
# # Convert Alpha to Z-score
# df['Z_Alpha'] = (df['Alpha'] - df['mean_Alpha']) / df['STD_Alpha']
# # Calculate Normalized Alpha Quality Score
# df['Alpha_Score'] = np.where(df['Z_Alpha'] >= 0, 
#                                      1 + df['Z_Alpha'], 
#                                      (1 - df['Z_Alpha'])**-1)
df
df[df['Date'] == '2025-02-17'].sort_values(by=['Date','Alpha'], ascending=[True,False]).head(25)

,Date,Symbol,Alpha
1497489,2025-02-17,DIXON,0.003111
1497615,2025-02-17,M&M,0.002126
1497488,2025-02-17,DIVISLAB,0.002092
1497680,2025-02-17,MUTHOOTFIN,0.001960
1497692,2025-02-17,ZOMATO,0.001680
1497717,2025-02-17,BHARTIARTL,0.001677
1497451,2025-02-17,BAJAJHLDNG,0.001592
1497764,2025-02-17,VEDL,0.001527
1497567,2025-02-17,INDUSTOWER,0.001469
1497551,2025-02-17,HINDZINC,0.001455


In [71]:
rebalanceFrequency=3
from rebalance import *
op=rebalanceDates(price_data,'2006-06-02',17, rebalanceFrequency)
rebalanceDates=op
df_rebalance_dates = pd.DataFrame({'Timestamp': pd.to_datetime(rebalanceDates)})
df_rebalance_dates.rename(columns={'Timestamp': 'Date'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])

df = df[df['Date'].isin(df_rebalance_dates['Date'])].reset_index(drop=True)
top_stocks = 20
df_stocks = df.copy()
df_stocks = df_stocks.groupby('Date').apply(lambda x: x.nlargest(top_stocks, 'Alpha')).reset_index(drop=True)

# # Normalize Alpha values to sum to 100 for each rebalance date
# df_stocks['Weight'] = (
#     df_stocks.groupby('Date')['Alpha']
#     .transform(lambda x: (x / x.sum()) * 100))

# ffweights = pd.read_csv('ffweights.csv')
# ffweights['Date'] = pd.to_datetime(ffweights['Date'])
# df_stocks = pd.merge(df_stocks,ffweights, on =['Date','Symbol'], how='left')
df_stocks.dropna().tail(20)

,Date,Symbol,Alpha
1480,2024-12-18,OFSS,0.003722
1481,2024-12-18,DIXON,0.003437
1482,2024-12-18,ZOMATO,0.003150
1483,2024-12-18,TRENT,0.003038
1484,2024-12-18,MAZDOCK,0.002545
1485,2024-12-18,PERSISTENT,0.002334
1486,2024-12-18,MAXHEALTH,0.002304
1487,2024-12-18,INDHOTEL,0.001937
1488,2024-12-18,TORNTPHARM,0.001863
1489,2024-12-18,VEDL,0.001785


In [72]:
x = df_stocks.dropna()
x.to_csv('Large-alpha.csv')

In [63]:
df_stocks.to_csv('Alpha210MC.csv', index=False)

In [73]:
churnDf = df_stocks.groupby("Date")["Symbol"].apply(set).rename("Current").reset_index()
churnDf["Previous"] = churnDf["Current"].shift()
churnDf = churnDf.dropna()
churnDf["Churn"] = churnDf.apply(lambda x : len(x["Current"] - x["Previous"])/len(x["Previous"]), axis = 1)
churnDf["Churn"].mean()

0.40878378378378377

In [75]:
df_stocks.dropna()

,Date,Symbol,Alpha
60,2007-03-19,ADANIENT,0.005262
61,2007-03-19,INDIABULLS,0.004141
62,2007-03-19,GODREJIND,0.002832
63,2007-03-19,JSWSTEEL,0.002509
64,2007-03-19,GLENMARK,0.002503
...,...,...,...
1495,2024-12-18,IRFC,0.001510
1496,2024-12-18,BOSCHLTD,0.001507
1497,2024-12-18,UNITDSPR,0.001455
1498,2024-12-18,BHARTIARTL,0.001405
